In [90]:
import pandas as pd
import numpy as np

data = pd.read_csv('iris.csv')

test_idx = np.random.choice(data.shape[0], int(data.shape[0]*0.9))
data_org = data.drop(test_idx)
data_test = data.iloc[test_idx]

In [78]:
import numpy as np
from functools import partial

def dist_Euclidean(a: np.ndarray, b: np.ndarray):
    return ((a-b)**2).sum()

def dist_Manhattan(a: np.ndarray, b: np.ndarray):
    return np.abs(a-b).sum()

def knn(data_population: pd.DataFrame, data_sample: pd.DataFrame, target_column, n, attr_columns=None, dist_func=dist_Euclidean):
    if attr_columns is None:
        attr_columns = data_sample.columns
        if target_column in attr_columns:
            attr_columns = attr_columns.drop(target_column)
    
    data_sample = data_sample[attr_columns]
    data_attrs = data_population[attr_columns]
    data_labels = data_population[target_column]

    def single_classify(sample):
        dist = data_attrs.apply(partial(dist_func, sample.reshape((-1,))), axis=1, raw=True)
        return pd.concat([dist.sort_values(), data_labels], axis=1).head(n)[target_column].value_counts(sort=True).index[0]
    return data_sample.apply(single_classify, raw=True, axis=1)



In [92]:
pred = knn(
    data_org, data_test, 'Species', 10,
    attr_columns=['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm'],
    dist_func=dist_Manhattan
    )
(pred == data_test['Species']).to_numpy().mean().round(2)

0.96